In [45]:
import numpy as np
import pandas as pd
import math

dataFrame = pd.read_csv("train.csv") 
ignorableColumns = ["PassengerId", "Survived", "Name", "Ticket", "Cabin"]
ageMean = dataFrame["Age"].mean()
dataFrame["Age"].fillna(ageMean, inplace=True)
dataFrame["Embarked"].fillna("NA", inplace=True)
data = dataFrame.values
columns = dataFrame.columns
methodDict = {'PassengerId': -1,
              'Survived': -1,
              'Pclass': 0,
              'Name': -1,
              'Sex': 0,
              'Age': 1,
              'SibSp': 1,
              'Parch': 1,
              'Ticket': -1,
              'Fare': 1,
              'Cabin': -1,
              'Embarked': 0}
methodStore = dict()
data = np.transpose(data)
data

array([[1, 2, 3, ..., 97, 98, 99],
       [0, 1, 1, ..., 0, 1, 1],
       [3, 1, 3, ..., 1, 1, 2],
       ...,
       [7.25, 71.2833, 7.925, ..., 34.6542, 63.3583, 23.0],
       [nan, 'C85', nan, ..., 'A5', 'D10 D12', nan],
       ['S', 'C', 'S', ..., 'C', 'C', 'S']], dtype=object)

In [46]:
def getIndexOfColumn(columnName, columns):
  for i in range(len(columns)):
    if(columns[i] == columnName):
      return i
  return -1

In [47]:
def makeDiffclass(data, columnName, columns):
  column = getIndexOfColumn(columnName, columns)
  Survived = getIndexOfColumn("Survived", columns)
  unique = dataFrame[columnName].unique()
  valsDict = dict()
  # for columns with unique values(fixed value)
  if(methodDict[columnName] == 0):
    dicArr = dict()
    for item in unique:
      dicArr[item] = [0,0]
    for i in range(len(data[column])):
      if(data[Survived][i] == 1):
        dicArr[data[column][i]][0] += 1
      else:
        dicArr[data[column][i]][1] += 1
    return dicArr
  # for columns with general numbers
  else:
    mean = dataFrame[columnName].mean()
    std = dataFrame[columnName].std()
    no_of_classes = 6
    methodStore[columnName] = {"mean":mean, "std":std, "no_of_classes":no_of_classes}
    valsDict = dict()
    for i in range(int(no_of_classes/2) + 1):
      # print(i)
      valsDict[i] = [0,0]
      valsDict[-i] = [0,0]
    valsDict[-(no_of_classes/2)-1] = [0,0]
    valsDict[+(no_of_classes/2)+1] = [0,0]
    for i in range(len(data[column])):
      itemNormalised = (data[column][i] - mean)/std
      itemClass = math.floor(itemNormalised*(no_of_classes/2))
      if(itemClass<-(no_of_classes/2)-1):
        data[column][i] = -math.floor(no_of_classes/2)-1
        if(data[Survived][i] == 1):
          valsDict[-math.floor(no_of_classes/2)-1][0] += 1
        else:
          valsDict[-math.floor(no_of_classes/2)-1][1] += 1
      elif(itemClass>(no_of_classes/2)):
        data[column][i] = +math.floor(no_of_classes/2)+1
        if(data[Survived][i] == 1):
          valsDict[+math.floor(no_of_classes/2)+1][0] += 1
        else:
          valsDict[+math.floor(no_of_classes/2)+1][1] += 1
      else:
        data[column][i] = itemClass
        if(itemClass < 0):
          itemClass += 1
        if(data[Survived][i] == 1):
          valsDict[itemClass][0] += 1
        else:
          valsDict[itemClass][1] += 1
    return valsDict

In [48]:
def changeAllColumns(dataFrame, data):
  for column in dataFrame:
    if(column in ignorableColumns):
      continue
    makeDiffclass(data, column, dataFrame.columns)
changeAllColumns(dataFrame, data)

In [49]:
def entropyWithConditions(data, conditions, columnName):
  column = getIndexOfColumn(columnName, columns)
  Survived = getIndexOfColumn("Survived", columns)
  
  dicArr = dict()
  for i in range(len(data[0])):
    yesRow = True
    for key in conditions.keys():
      # print(data[key][i], conditions[key])
      if(data[key][i] != conditions[key]):
        yesRow = False
        break
    if(yesRow):
      if(data[Survived][i] == 1):
        if(data[column][i] in dicArr.keys()):
          dicArr[data[column][i]][0] += 1
        else:
          dicArr[data[column][i]] = [1,0]
      else:
        if(data[column][i] in dicArr.keys()):
          dicArr[data[column][i]][1] += 1
        else:
          dicArr[data[column][i]] = [0,1]
  return dicArr

In [50]:
import math
def findEntropy(data, conditions, column):
  dictClasses = entropyWithConditions(data, conditions, column)
  if(len(dictClasses.keys()) == 0):
    return float('inf'), [1,0]
  total = 0
  entropy = 0
  for key in dictClasses.keys():
    entA = dictClasses[key][0]
    entB = dictClasses[key][1]
    if(entA == 0 or entB == 0):
      tempEntropy = 0
    else:
      tempEntropy = (entA/(entA+entB))*math.log(entA/(entA+entB))
    entropy += tempEntropy*(entA + entB)
    total += (entA + entB)
  entropy /= total
  sur = [0,0]
  for key in dictClasses.keys():
    sur[0]+=dictClasses[key][0]
    sur[1]+=dictClasses[key][1]
  return entropy, sur

In [51]:
ageColumn = getIndexOfColumn('Age', columns)
conditions = {
    ageColumn:2
}
findEntropy(data, conditions, ageColumn)

(-0.2986265782046758, [3, 3])

In [52]:
class Node:
   def __init__(self, currentlyChecking = None, conditions = None, next = None, ignore = ignorableColumns, survival = [1,0]):
      self.currentlyChecking = currentlyChecking
      self.conditions = conditions
      self.next = next
      self.survival = survival
      self.ignore = ignore

class Tree:
   def __init__(self):
      self.headNode = None


In [53]:
tree = Tree()
conditions = dict()
next = dict()
startNode = Node("", conditions, next)
tree.headNode = startNode
q = [startNode]

while(len(q)>0):
  p = q.pop(0)
  ignore = p.ignore
  bestColumn = ""
  bestSurvival = []
  leastEntropy = float("inf")
  for column in columns:
    if(column in ignore):
      continue
    # print(column+" ", end='')
    temp, survival = findEntropy(data, p.conditions, column)
    if(temp < leastEntropy):
      leastEntropy = temp
      bestColumn = column
      bestSurvival = survival
  if(bestColumn == ""):
    continue
  p.currentlyChecking = bestColumn
  p.survival = bestSurvival
  bestColumnNumber = getIndexOfColumn(p.currentlyChecking, columns)
  unique = np.unique(data[bestColumnNumber])
  for item in unique:
    newConditions = p.conditions.copy()
    newIgnore = ignore.copy()
    newIgnore.append(bestColumn)
    if(len(newIgnore) == 12):
      continue
    newConditions[bestColumnNumber] = item
    newNode = Node(None, newConditions, dict(), newIgnore)
    p.next[item] = newNode
    q.append(newNode)
  # print(p.next)

**Testing Part**

In [54]:
# row = [2, ]
# def findProbOfLivingSolo(row):
print(methodStore)

{'Age': {'mean': 27.380909090909086, 'std': 13.526974878022408, 'no_of_classes': 6}, 'SibSp': {'mean': 0.7272727272727273, 'std': 1.185096067906301, 'no_of_classes': 6}, 'Parch': {'mean': 0.4444444444444444, 'std': 0.9712418121129115, 'no_of_classes': 6}, 'Fare': {'mean': 29.553156565656565, 'std': 41.17987152284466, 'no_of_classes': 6}}


In [55]:
dataFrame = pd.read_csv("test.csv") 
ignorableColumns = ["PassengerId", "Survived", "Name", "Ticket", "Cabin"]
ageMean = dataFrame["Age"].mean()
dataFrame["Age"].fillna(ageMean, inplace=True)
dataFrame["Embarked"].fillna("NA", inplace=True)
fareMean = dataFrame["Fare"].mean()
dataFrame["Fare"].fillna(fareMean, inplace=True)
data = dataFrame.values
columns = dataFrame.columns
data = np.transpose(data)

In [56]:
def makeDiffclass2(data, columnName, columns):
  column = getIndexOfColumn(columnName, columns)
  Survived = getIndexOfColumn("Survived", columns)
  unique = dataFrame[columnName].unique()
  valsDict = dict()
  # for columns with unique values(fixed value)
  if(methodDict[columnName] == 0):
    dicArr = dict()
    for item in unique:
      dicArr[item] = [0,0]
    for i in range(len(data[column])):
      if(data[Survived][i] == 1):
        dicArr[data[column][i]][0] += 1
      else:
        dicArr[data[column][i]][1] += 1
    return dicArr
  # for columns with general numbers
  else:
    mean = methodStore[columnName]["mean"]
    std = methodStore[columnName]["std"]
    no_of_classes = methodStore[columnName]["no_of_classes"]
    methodStore[columnName] = {mean:mean, std:std, no_of_classes:no_of_classes}
    valsDict = dict()
    for i in range(int(no_of_classes/2) + 1):
      # print(i)
      valsDict[i] = [0,0]
      valsDict[-i] = [0,0]
    valsDict[-(no_of_classes/2)-1] = [0,0]
    valsDict[+(no_of_classes/2)+1] = [0,0]
    for i in range(len(data[column])):
      itemNormalised = (data[column][i] - mean)/std
      itemClass = math.floor(itemNormalised*(no_of_classes/2))
      if(itemClass<-(no_of_classes/2)-1):
        data[column][i] = -math.floor(no_of_classes/2)-1
        if(data[Survived][i] == 1):
          valsDict[-math.floor(no_of_classes/2)-1][0] += 1
        else:
          valsDict[-math.floor(no_of_classes/2)-1][1] += 1
      elif(itemClass>(no_of_classes/2)):
        data[column][i] = +math.floor(no_of_classes/2)+1
        valsDict[+math.floor(no_of_classes/2)+1][0] += 1
      else:
        data[column][i] = itemClass
        if(itemClass < 0):
          itemClass += 1
        valsDict[itemClass][0] += 1
    return valsDict

In [57]:
def changeAllColumns2(dataFrame, data):
  print("ignorableColumns",ignorableColumns)
  for column in dataFrame:
    if(column in ignorableColumns):
      continue
    print(methodStore)
    makeDiffclass2(data, column, dataFrame.columns)
changeAllColumns2(dataFrame, data)
data

ignorableColumns ['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin']
{'Age': {'mean': 27.380909090909086, 'std': 13.526974878022408, 'no_of_classes': 6}, 'SibSp': {'mean': 0.7272727272727273, 'std': 1.185096067906301, 'no_of_classes': 6}, 'Parch': {'mean': 0.4444444444444444, 'std': 0.9712418121129115, 'no_of_classes': 6}, 'Fare': {'mean': 29.553156565656565, 'std': 41.17987152284466, 'no_of_classes': 6}}
{'Age': {'mean': 27.380909090909086, 'std': 13.526974878022408, 'no_of_classes': 6}, 'SibSp': {'mean': 0.7272727272727273, 'std': 1.185096067906301, 'no_of_classes': 6}, 'Parch': {'mean': 0.4444444444444444, 'std': 0.9712418121129115, 'no_of_classes': 6}, 'Fare': {'mean': 29.553156565656565, 'std': 41.17987152284466, 'no_of_classes': 6}}
{'Age': {'mean': 27.380909090909086, 'std': 13.526974878022408, 'no_of_classes': 6}, 'SibSp': {'mean': 0.7272727272727273, 'std': 1.185096067906301, 'no_of_classes': 6}, 'Parch': {'mean': 0.4444444444444444, 'std': 0.9712418121129115, 'no_of_classe

array([[101, 102, 103, ..., 889, 890, 891],
       [0, 0, 0, ..., 0, 1, 0],
       [3, 3, 1, ..., 3, 1, 3],
       ...,
       [-2, -2, 3, ..., -1, 0, -2],
       [nan, nan, 'D26', ..., nan, 'C148', nan],
       ['S', 'S', 'S', ..., 'S', 'C', 'Q']], dtype=object)

In [58]:
# to look at the tree
def dfs(node, string):
  string = string + " " +str(node.currentlyChecking)
  print(string)
  print(len(node.next.keys()))
  for key in node.next.keys():
    # print("here", node.next[key])
    if(node.next[key].currentlyChecking == None):
      print("///////")
      break
    dfs(node.next[key], string)

dfs(tree.headNode, "")

 Parch
3
 Parch Pclass
3
 Parch Pclass SibSp
4
 Parch Pclass SibSp Age
9
///////
 Parch Pclass SibSp Embarked
4
 Parch Pclass SibSp Embarked Fare
7
///////
///////
///////
 Parch Pclass SibSp
4
 Parch Pclass SibSp Embarked
4
///////
 Parch Pclass SibSp Sex
2
 Parch Pclass SibSp Sex Fare
7
///////
///////
///////
 Parch Pclass Fare
7
 Parch Pclass Fare SibSp
4
 Parch Pclass Fare SibSp Age
9
///////
 Parch Pclass Fare SibSp Sex
2
 Parch Pclass Fare SibSp Sex Age
9
///////
 Parch Pclass Fare SibSp Sex Age
9
///////
 Parch Pclass Fare SibSp Sex
2
///////
///////
 Parch Pclass Fare Sex
2
 Parch Pclass Fare Sex Embarked
4
///////
 Parch Pclass Fare Sex Age
9
///////
///////
 Parch Sex
2
 Parch Sex Fare
7
///////
 Parch Sex Pclass
3
 Parch Sex Pclass SibSp
4
 Parch Sex Pclass SibSp Fare
7
///////
///////
///////
 Parch Embarked
4
 Parch Embarked Pclass
3
///////
///////


In [59]:
def findPrbOfLife(randomRow = 123, maxDepth = 3):
  curr = tree.headNode
  for i in range(maxDepth):
    # print(curr.currentlyChecking)
    columnName = curr.currentlyChecking
    columnNumber = getIndexOfColumn(columnName, columns)
    if(len(curr.next.keys()) == 0):
      break
    curr = curr.next[data[columnNumber][randomRow]]

  a = curr.survival
  return a[0]/(a[0] + a[1])
  print("survival chances = ", str(a[0])+'/'+str(a[0]+a[1]))

In [60]:
findPrbOfLife()

0.3

In [68]:
def checkWithDepth(depth = 3):
  truePositive = 0
  trueNegative = 0
  falsePositive = 0
  falseNegative = 0
  Survived = getIndexOfColumn("Survived", columns)
  for i in range(len(data[0])):
    try:
      prob = findPrbOfLife(i, depth)
    except:
      # print(i)
      continue
    if(prob > 1/2):
      if(data[Survived][i]==1):
        truePositive+=1
      else:
        falsePositive+=1
    else:
      if(data[Survived][i]==0):
        trueNegative+=1
      else:
        falseNegative+=1

  total = truePositive + falsePositive + trueNegative + falseNegative
  print("accuracy = ",(truePositive + trueNegative)/total)

In [69]:
for i in range(10):
  print("depth = ",i)
  checkWithDepth(i)

depth =  0
accuracy =  0.6194690265486725
depth =  1
accuracy =  0.6194690265486725
depth =  2
accuracy =  0.6131479140328698
depth =  3
accuracy =  0.6238881829733164
depth =  4
accuracy =  0.6607369758576874
depth =  5
accuracy =  0.6645489199491741
depth =  6
accuracy =  0.6122448979591837
depth =  7
accuracy =  0.6122448979591837
depth =  8
accuracy =  0.6122448979591837
depth =  9
accuracy =  0.6122448979591837
